In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (12, 8)
plt.rcParams["font.size"] = 12

In [ ]:
import inspect

import xarray as xr
import xarray_sentinel

from sarsen import apps, geocoding, orbit, scene

# uncomment to check that the code below is in sync with the implementation
# print(inspect.getsource(apps.backward_geocode_sentinel1_slc_iw_burst))

In [ ]:
product_path = (
    "data/S1B_IW_GRDH_1SDV_20211223T051122_20211223T051147_030148_039993_5371.SAFE"
)
dem_urlpath = "data/Rome-10m-DEM.tif"
output_urlpath = "Rome-10m-TC-GRD.tif"
swath_group = "IW"
swath_polarisation_group = "IW/VV"
measurement_group = "IW/VV"
measurement_block_slices = (slice(7000, 9000), slice(20000, 23000))
digital_number_max = 600

!ls -d {product_path}

In [ ]:
measurement = xr.open_dataset(
    product_path, engine="sentinel-1", group=measurement_group
)
measurement

In [ ]:
measurement_block = measurement.measurement[measurement_block_slices]
measurement_block

In [ ]:
_ = abs(measurement_block).plot(y="azimuth_time", vmax=digital_number_max)

In [ ]:
orbit_ecef = xr.open_dataset(
    product_path, engine="sentinel-1", group=f"{swath_polarisation_group}/orbit"
)
orbit_ecef

In [ ]:
dem_raster = scene.open_dem_raster(dem_urlpath)
print(dem_raster.rio.estimate_utm_crs())
dem_raster

In [ ]:
_ = dem_raster.plot(figsize=(12, 8))

In [ ]:
try:
    dem_ecef = scene.convert_to_dem_ecef(dem_raster)
except:
    # first time we run it we get a GDAL exception :shrug:
    dem_ecef = scene.convert_to_dem_ecef(dem_raster)
dem_ecef

In [ ]:
orbit_interpolator = orbit.OrbitPolyfitIterpolator.from_position(orbit_ecef.position)
orbit_interpolator.interval

In [ ]:
position_sar = orbit_interpolator.position(measurement.azimuth_time)
velocity_sar = orbit_interpolator.velocity(measurement.azimuth_time)

position_sar

In [ ]:
dem_coords = geocoding.backward_geocode(dem_ecef, position_sar, velocity_sar)
dem_coords

In [ ]:
coordinate_conversion = xr.open_dataset(
    product_path,
    engine="sentinel-1",
    group=f"{swath_polarisation_group}/coordinate_conversion",
)
coordinate_conversion

In [ ]:
slant_range = geocoding.SPEED_OF_LIGHT / 2.0 * dem_coords.slant_range_time
cc = coordinate_conversion.interp(azimuth_time=dem_coords.azimuth_time)
x = slant_range - cc.sr0
ground_range = (cc.srgrCoefficients * x ** cc.degree).sum("degree")
ground_range

In [ ]:
geocoded = abs(measurement.measurement).sel(
    azimuth_time=dem_coords.azimuth_time,
    ground_range=ground_range,
    method="nearest",
)
geocoded.rio.set_crs(dem_raster.rio.crs)
geocoded

In [ ]:
_ = geocoded.plot(vmax=digital_number_max, figsize=(12, 8))

In [ ]:
geocoded.rio.to_raster(
    output_urlpath,
    tiled=True,
    blockxsize=512,
    blockysize=512,
    compress="DEFLATE",
    num_threads="ALL_CPUS",
)